<a href="https://colab.research.google.com/github/swguo/Generating-Personalized-Phishing-Emails/blob/main/GPT2_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%time
%%capture
!pip install transformers==4.5.0

CPU times: user 68 ms, sys: 17.4 ms, total: 85.3 ms
Wall time: 9.18 s


In [3]:
!nvidia-smi

Wed Mar 16 07:02:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
import os
import io
import requests
import numpy as np
import pandas as pd
import re
import zipfile
import random
import time
import csv
import datetime
from itertools import compress
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, \
                         Trainer, TrainingArguments

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler
import pprint
pp = pprint.PrettyPrinter(indent=4)
from IPython.display import clear_output

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(f"PyTorch version: {torch.__version__}")

PyTorch version: 1.10.0+cu111


# Configurations

In [16]:
DEBUG           = False

INPUT_DIR       = 'articles'

USE_APEX        = True
APEX_OPT_LEVEL  = 'O1'

MODEL           = 'gpt2-medium' #{gpt2, gpt2-medium, gpt2-large, gpt2-xl}

UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training

SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}
                    
MAXLEN          = 768  #{768, 1024, 1280, 1600}

TRAIN_SIZE      = 0.8

if USE_APEX:
    TRAIN_BATCHSIZE = 4
    BATCH_UPDATE    = 16
else:
    TRAIN_BATCHSIZE = 2
    BATCH_UPDATE    = 32

EPOCHS          = 4
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1e2

SEED            = 2020

In [17]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

# Loading Tokenizer, Config and Model

In [18]:
def get_tokenier(special_tokens=None):
    tokenizer = AutoTokenizer.from_pretrained(MODEL) #GPT2Tokenizer

    if special_tokens:
        tokenizer.add_special_tokens(special_tokens)
        print(len(tokenizer))
        print("Special tokens added")
    return tokenizer

def get_model(tokenizer, special_tokens=None, load_model_path=None):

    #GPT2LMHeadModel
    if special_tokens:
        config = AutoConfig.from_pretrained(MODEL, 
                          bos_token_id=tokenizer.bos_token_id,
                          eos_token_id=tokenizer.eos_token_id,
                          sep_token_id=tokenizer.sep_token_id,
                          pad_token_id=tokenizer.pad_token_id,
                          output_hidden_states=False)
    else: 
        config = AutoConfig.from_pretrained(MODEL,                                     
                          pad_token_id=tokenizer.eos_token_id,
                          output_hidden_states=False)    

    #----------------------------------------------------------------#
    model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

    if special_tokens:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

    if load_model_path:
        model.load_state_dict(torch.load(load_model_path))

    model.cuda()
    return model

In [19]:
def join_keywords(keywords, randomize=True):
        N = len(keywords)

        #random sampling and shuffle
        if randomize: 
            M = random.choice(range(N+1))
            keywords = keywords[:M]
            random.shuffle(keywords)

        return ','.join(keywords)

# Download Spear Email model - GPT2


In [10]:
!pip install --upgrade --no-cache-dir gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=6b3ea9bf422a4f38d555a8f4b9e13d324377f2ea03c7a897d33fe617905ab088
  Stored in directory: /tmp/pip-ephem-wheel-cache-rd2e7hud/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [11]:
!gdown --id 1dPhbShbTwBB_3cdsV4zp0gLFD0mnREIO

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1dPhbShbTwBB_3cdsV4zp0gLFD0mnREIO
To: /content/pytorch_model.bin
100% 1.44G/1.44G [00:45<00:00, 31.6MB/s]


In [20]:
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)

model = get_model(tokenizer, 
          special_tokens=SPECIAL_TOKENS,
          load_model_path='pytorch_model.bin')

50262
Special tokens added


Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Title => 文體起始句

type => 類型
 * Fraud  : 詐騙/犯罪類型
 * Normal : 一般郵件

category => 內文主題
 * 'CRIME' ,'ENTERTAINMENT', 'POLITICS', 'SPORTS', 'BUSINESS', 'TECH', 'EDUCATION',
 'HEALTHY LIVING', 'MONEY', 'CULTURE & ARTS'

formats => 文體撰寫手法 
 * News 新聞
 * Email 郵件

keywords => 內文希望出現的關鍵字

# Generation

## Define Function

In [21]:
import re
def remove_Symbol(s):
    s = re.sub(r'[^\w]','',s)
    return s


In [22]:
def generation_token(title,text,keywords,types,category,formats):    
    
    kw = join_keywords(keywords, randomize=False)
    prompt = SPECIAL_TOKENS['bos_token'] + types + \
             SPECIAL_TOKENS['sep_token'] + category + \
             SPECIAL_TOKENS['sep_token'] + formats + \
             SPECIAL_TOKENS['sep_token'] + title + \
             SPECIAL_TOKENS['sep_token'] + kw + \
             SPECIAL_TOKENS['sep_token']
    
    prompt_len = len(types)+len(category)+len(formats)+ len(title) + len(','.join(keywords)) 
    
    generated = tokenizer([prompt],max_length=MAXLEN, return_tensors='pt')['input_ids'].to(device)
    

    model.eval()

    sample_outputs = model.generate(
        generated,
        do_sample=False, 
        max_length=MAXLEN, 
        top_k=1,
        top_p=0.75,
        num_return_sequences=3
    )
    
    return sample_outputs,len(prompt)

## 1. Generation phishing emails for fraud

## Topic for COIVD19 of Fraud Emails

In [55]:
title = "please reply to me soon "
#title = "please complete as soon as possible "
#title = 'I will get back to you in the am '
#title = 'We are keenly interested in setting up a new hospital '
#title = 'How about was that payment? reply to me soon '
#title = "I will get back to you in the am "

keywords = ['covid19', 'case', 'hospital','CDC']

In [56]:
types = "Fraud"
category = "BUSINESS"
formats = "Email"
kw = join_keywords(keywords, randomize=False)
prompt = SPECIAL_TOKENS['bos_token'] + types + \
                SPECIAL_TOKENS['sep_token'] + category + \
                SPECIAL_TOKENS['sep_token'] + formats + \
                SPECIAL_TOKENS['sep_token'] + title + \
                SPECIAL_TOKENS['sep_token'] + kw
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval();

In [58]:
from termcolor import colored
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                do_sample=True,   
                min_length=20, 
                max_length=200,
                top_k=10,                              
                top_p=0.5,
                repetition_penalty=2.0,
                num_return_sequences=3
                )

predt_email = []
prompt = len(types)+len(category)+len(formats)+ len(title) + len(kw) 
for i, sample_output in enumerate(sample_outputs):
  predit_text = tokenizer.decode(sample_output, skip_special_tokens=True)[prompt:]
  index = predit_text.find('.')
  result = " ".join(colored(t,'white','on_red') if remove_Symbol(t) in keywords else remove_Symbol(t) for t in predit_text[index+1:].lower().split())
  print('Phishing Email : {}'.format(i+1))
  print("="*100)
  for j,t in enumerate(result.split(' ')):
    print(t+' ',end='')
    if j%20==0 and j!=0:
      print('\n')
  print('\n')

Phishing Email : 1
i will be very glad if you can give your time and attention for a mutually beneficial business venturei am drdaniel 

j the director in charge of auditing  accounting section at cdcnyscavid19casedrauditorsdealingbankbankplease please reply to me soon 

Phishing Email : 2
i am drdavid covids a senior staff in the cdcs office of medical records and information technology mdr sectioni have urgent 

need for an immediate foreign partner who can assist us with this transaction as we are still recovering from our 

recent hospitalization due on covid-19. my name is david cevides mdthe chief executive officerof dbe international consortium ltdcavid20 please reply 

urgently to me soon 

Phishing Email : 3
joseph_mccoyyahoocomi am contacting you on a business transaction that will be of great interest and reward for bothof us i have 

been diagnosed with covid19 a rare genetic condition which requires immediate medical attention and hospitalizationcavidsdocumentedbankdrjosep

## Topic for Finace of Fraud Emails

In [59]:
title = 'How was that payment? reply to me soon '
#title = 'I will get back to you in the am '
#title = 'please, reply to me soon '
keywords = ['bank','account','number','reply','credit']

In [60]:
types = "Fraud" 
category = "BUSINESS"
formats = "Email"
kw = join_keywords(keywords, randomize=False)


prompt = SPECIAL_TOKENS['bos_token'] + types + \
                SPECIAL_TOKENS['sep_token'] + category + \
                SPECIAL_TOKENS['sep_token'] + formats + \
                SPECIAL_TOKENS['sep_token'] + title + \
                SPECIAL_TOKENS['sep_token'] + kw
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval();

In [49]:
from termcolor import colored
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                do_sample=True,   
                min_length=20, 
                max_length=200,
                top_k=10,                              
                top_p=0.5,
                repetition_penalty=2.0,
                num_return_sequences=3
                )

predt_email = []
prompt = len(types)+len(category)+len(formats)+ len(title) + len(kw) 
for i, sample_output in enumerate(sample_outputs):
  predit_text = tokenizer.decode(sample_output, skip_special_tokens=True)[prompt:]
  index = predit_text.find('.')
  result = " ".join(colored(t,'white','on_red') if remove_Symbol(t) in keywords else remove_Symbol(t) for t in predit_text[index+1:].lower().split())
  print('Phishing Email : {}'.format(i+1))
  print("="*100)
  for j,t in enumerate(result.split(' ')):
    print(t+' ',end='')
    if j%20==0 and j!=0:
      print('\n')
  print('\n')

Phishing Email : 1
i am contacting you because i want your help and assistance in a matter of timemy name is john doe and 

i work with the international operation departmentof one foreign firm in nigeriabankaccountsjohnafricahow was that payment made for credit reasons or 

on behalf thereof reply my email belowhttpmail2yahoocommail_promo1html dear friendplease permit this brief introduction before we proceedbankaddressbanka0why did he pay his 

bill through an account opened by him without prior consent from us please be patient as it takes sometimefor our 

bank director the manager who will have all necessary information regardingthis transaction sothatyou can make sure everything goes wellwith 

Phishing Email : 2
i am contacting you because of a financial emergency and the need for immediate assistance from my bank in regardsof this 

accountwhich is presently pending or any other fundsthat may beinvolving itplease do not hesitate at usingthis medium asap to reach 

outfor help a

## 2. Generation phishing emails for BEC

## Topic for COIVD19 of BEC Emails

In [61]:
title = "please reply to me soon "
#title = "please complete as soon as possible "
#title = 'I will get back to you in the am '
#title = 'We are keenly interested in setting up a new hospital '
#title = 'How about was that payment? reply to me soon '
#title = "I will get back to you in the am "
types = "Normal" #犯罪的文章，包含詐騙郵件詞表
#types = "Normal" 
category = "BUSINESS" #商務文章，包含正常郵件詞表 
formats = "Email" #Email文體

#情境1 面向職員秘書助理
#keywords = ['read', 'bank', 'account', 'reply']
#keywords = ['officer', 'attached', 'account','bank','credit']
#keywords = ['officer', 'attached', 'amount','bank','credit']
#keywords = ['bank','account','reply','number','credit']
#情境2 面向醫師高階管職
keywords = ['covid19', 'case', 'hospital','CDC']
kw = join_keywords(keywords, randomize=False)


prompt = SPECIAL_TOKENS['bos_token'] + types + \
                SPECIAL_TOKENS['sep_token'] + category + \
                SPECIAL_TOKENS['sep_token'] + formats + \
                SPECIAL_TOKENS['sep_token'] + title + \
                SPECIAL_TOKENS['sep_token'] + kw
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval();

In [64]:
from termcolor import colored
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                do_sample=True,   
                min_length=20, 
                max_length=200,
                top_k=10,                              
                top_p=0.5,
                repetition_penalty=2.0,
                num_return_sequences=3
                )

predt_email = []
prompt = len(types)+len(category)+len(formats)+ len(title) + len(kw) 
for i, sample_output in enumerate(sample_outputs):
  predit_text = tokenizer.decode(sample_output, skip_special_tokens=True)[prompt:]
  index = predit_text.find('.')
  result = " ".join(colored(t,'white','on_red') if remove_Symbol(t) in keywords else remove_Symbol(t) for t in predit_text[index+1:].lower().split())
  print('Phishing Email : {}'.format(i+1))
  print("="*100)
  for j,t in enumerate(result.split(' ')):
    print(t+' ',end='')
    if j%20==0 and j!=0:
      print('\n')
  print('\n')

Phishing Email : 1
i will be in dc until monday at which time you can send the letter and well discuss how best proceed 

with a formalletterconfirmation that is bothlegal routine for us as wellasfor your colleagues on capitol hill 

Phishing Email : 2
im in the hospital recovering from a stroke and am on ventilators for at least three days the doctors are treating 

it as an acute case of pneumonia so theyre trying everything possible to keep him alive 

Phishing Email : 3
4please respondtome now sooner i will have a new draft of the letter that you can send in my direction asap 

and then we could discuss it further with him at his hospital room on thursday morning afternoon before he goes 

home for lunchtime or dinner time so i dont wantyou wasting your energy trying this again if there is something 

else goingon here and hope everyone getswell after all 



## Topic for finance of BEC Emails

In [65]:
title = 'How was that payment? reply to me soon '
#title = 'I will get back to you in the am '
#title = 'please, reply to me soon '
keywords = ['bank','account','number','reply','credit']
#keywords = ['bank','account','reply','number','credit']

In [73]:
types = "Normal" #犯罪的文章，包含詐騙郵件詞表
#types = "Normal" 
category = "BUSINESS" #商務文章，包含正常郵件詞表 
formats = "Email" #Email文體

kw = join_keywords(keywords, randomize=False)


prompt = SPECIAL_TOKENS['bos_token'] + types + \
                SPECIAL_TOKENS['sep_token'] + category + \
                SPECIAL_TOKENS['sep_token'] + formats + \
                SPECIAL_TOKENS['sep_token'] + title + \
                SPECIAL_TOKENS['sep_token'] + kw
print(prompt)
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)
print(generated)
model.eval();

<|BOS|>Normal<|SEP|>BUSINESS<|SEP|>Email<|SEP|>How was that payment? reply to me soon <|SEP|>bank,account,number,reply,credit
tensor([[50257, 26447, 50261, 45346, 44180, 50261, 15333, 50261,  2437,   373,
           326,  6074,    30, 10971,   284,   502,  2582,   220, 50261, 17796,
            11, 23317,    11, 17618,    11, 47768,    11, 43082]],
       device='cuda:0')


In [77]:
from termcolor import colored
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                do_sample=True,   
                min_length=20, 
                max_length=200,
                top_k=10,                              
                top_p=0.5,
                repetition_penalty=2.0,
                num_return_sequences=3
                )

predt_email = []
prompt = len(types)+len(category)+len(formats)+ len(title) + len(kw) 
for i, sample_output in enumerate(sample_outputs):
  predit_text = tokenizer.decode(sample_output, skip_special_tokens=True)[prompt:]
  index = predit_text.find('.')
  result = " ".join(colored(t,'white','on_red') if remove_Symbol(t) in keywords else remove_Symbol(t) for t in predit_text[index+1:].lower().split())
  print('Phishing Email : {}'.format(i+1))
  print("="*100)
  for j,t in enumerate(result.split(' ')):
    print(t+' ',end='')
    if j%20==0 and j!=0:
      print('\n')
  print('\n')

Phishing Email : 1
and my answer is simple its because we are dealing with an account number of 20 million dollars which has no 

interest rate on its face 

Phishing Email : 2
reply and we can discuss how bestto pay this bill or if you want us go after another lenderbusiness which is 

more likely to be of interest than your current financial situation 

Phishing Email : 3
i have not yet heard from them but if there is any way forward we will be very gratefulif one can 

come along who hasthe capacityinterests necessary at our disposal to assist us 

